In [1]:
import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
import os

TRAINING = True

In [2]:
with open('moves.txt', 'r') as infile:
    moves = infile.read()

In [3]:
vocab = sorted(set(moves))

print(f'{len(vocab)} unique characters')

33 unique characters


In [4]:
ids_from_chars = preprocessing.StringLookup(
    vocabulary=list(vocab), mask_token=None
)

chars_from_ids = tf.keras.layers.experimental.preprocessing.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

In [5]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

In [6]:
all_ids = ids_from_chars(tf.strings.unicode_split(moves, 'UTF-8'))

In [7]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

print(all_ids)

# for ids in ids_dataset.take(222):
#     print(chars_from_ids(ids).numpy().decode('utf-8'))

print(ids_dataset)

tf.Tensor([24  8  6 ...  2  2  1], shape=(3283587,), dtype=int64)
<TensorSliceDataset shapes: (), types: tf.int64>


In [8]:
seq_length = 397
examples_per_epoch = len(moves)

sequences = ids_dataset.batch(seq_length, drop_remainder=True)

In [9]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [10]:
dataset = sequences.map(split_input_target)

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module 'gast' has no attribute 'Index'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


In [11]:
BATCH_SIZE = 100
BUFFER_SIZE = 10000

dataset = (dataset.batch(BATCH_SIZE, drop_remainder=True).prefetch(tf.data.experimental.AUTOTUNE))

# for i in dataset.as_numpy_iterator():
#     for arr in i:
#         print(text_from_ids(arr).numpy())

In [12]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

In [13]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
    self.gru = tf.keras.layers.GRU(rnn_units, return_sequences=True, return_state=True)
    self.dense = tf.keras.layers.Dense(vocab_size)

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense(x, training=training)

    if return_state:
      return x, states
    else:
      return x

In [14]:
model = MyModel(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units
)

In [15]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

2021-10-23 13:22:29.616101: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-23 13:22:29.616245: W tensorflow/core/platform/profile_utils/cpu_utils.cc:126] Failed to get CPU frequency: 0 Hz


(100, 396, 34) # (batch_size, sequence_length, vocab_size)


In [16]:
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  8704      
_________________________________________________________________
gru (GRU)                    multiple                  3938304   
_________________________________________________________________
dense (Dense)                multiple                  34850     
Total params: 3,981,858
Trainable params: 3,981,858
Non-trainable params: 0
_________________________________________________________________


In [17]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
mean_loss = example_batch_loss.numpy().mean()
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("Mean loss:        ", mean_loss)

tf.exp(mean_loss).numpy()

Prediction shape:  (100, 396, 34)  # (batch_size, sequence_length, vocab_size)
Mean loss:         3.5263016


33.997997

In [18]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [19]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [20]:
EPOCHS = 120

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/120
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: unsupported operand type(s) for -: 'NoneType' and 'int'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
82/82 [==============================] - 2119s 26s/step - loss: 2.9003 - accuracy: 0.3454
Epoch 2/120
82/82 [==============================] - 2174s 26s/step - loss: 1.1193 - accuracy: 0.5966
Epoch 3/120
82/82 [==============================] - 2161s 26s/step - loss: 0.8070 - accuracy: 0.7215
Epoch 4/120
82/82 [==============================] - 2296s 2